In [ ]:
pip install Flask

In [19]:
from flask import Flask, render_template, request, url_for
import os

In [20]:
host_addr = "0.0.0.0"
port_num = "5050"

In [52]:
app = Flask(__name__)


# 웹 페이지 접속시 메인 화면
@app.route('/',  methods=['GET'])
def main_loading():
    mask = "img/mask2.png"
    return render_template("main.html", mask = mask)

# 파일 업로드 및 해당 파일 화면에 출력
@app.route('/',  methods=['GET', 'POST'])
def upload_file():
    # 파일 정보 받아오기
    age = request.form["age"]
    gender = request.form["gender"]
    audio_src = request.form["audio_src"]
    healthy = request.form.getlist("healty[]")
    print(healthy)
    
    
    #저장한 파일 화면에 출력하기
    return render_template("result.html", age=age)

if __name__ == "__main__":
    app.run()  

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [13/Jun/2022 00:59:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 00:59:11] "GET /static/img/mask2.png HTTP/1.1" 304 -
127.0.0.1 - - [13/Jun/2022 00:59:17] "POST / HTTP/1.1" 200 -


[]
